In [1]:
import sys
sys.path.append('../')


import numpy as np

class RegionNode:
    def __init__(self, idx, layer, volume, centroid, activation, is_bounded):
        # --- 1. Basic Metadata (Loaded from HDF5) ---
        self.id = idx
        self.layer = layer
        self.volume = volume          # Matches Julia: region.volume
        self.is_bounded = is_bounded  # Matches Julia: region.bounded
        self.centroid = centroid      # Matches Julia: region.x
        self.activation = activation  # Matches Julia: region.qlw
        
        # --- 2. Tree Topology ---
        self.parent = None
        self.children = []
        
        # --- 3. Geometric Matrices (Computed via hydration) ---
        # Matches Julia: region.Alw, region.clw
        self.Alw = None 
        self.clw = None
        
        # Matches Julia: region.Dlw, region.glw
        self.Dlw = None 
        self.glw = None 

    def get_path_inequalities(self):
        """
        Traverses up to the root to collect all constraints defining this region.
        Matches Julia: get_path_inequalities(r)
        """
        if self.Dlw is None:
            raise ValueError("Geometry not hydrated. Run hydrate_tree_geometry() first.")

        D_list = []
        g_list = []
        node = self
        
        while node is not None:
            D_list.append(node.Dlw)
            g_list.append(node.glw)
            node = node.parent
            
        # Stack Top-Down (Root -> Leaf)
        return np.vstack(D_list[::-1]), np.concatenate(g_list[::-1])

    def __repr__(self):
        """
        Matches Julia's Base.show output format
        Example: Region (L3) | Vol: 0.1234 | Act: [1 0 1] | Children: 2
        """
        vol_str = "Inf" if self.volume == float('inf') else f"{self.volume:.4f}"
        return (f"Region (L{self.layer}) | Vol: {vol_str} | "
                f"Bnd: {self.is_bounded} | Act: {self.activation} | "
                f"Children: {len(self.children)}")

def hydrate_tree_geometry(root, weights, biases, input_dim):
    """
    Reconstructs the Alw, clw, Dlw, glw matrices for the entire tree
    using the model weights and the node activation patterns.
    """
    # 1. Initialize Root Geometry (Standard Hypercube or Domain)
    root.Alw = np.eye(input_dim)
    root.clw = np.zeros(input_dim)
    
    # Root constraints: -1 <= x <= 1
    root.Dlw = np.vstack([np.eye(input_dim), -np.eye(input_dim)])
    root.glw = np.ones(2 * input_dim)
    
    queue = [root]
    
    while queue:
        parent = queue.pop(0)
        
        if not parent.children:
            continue
            
        # Get weights for transition: Layer L -> Layer L+1
        # (Julia layers are 1-based, Python lists 0-based)
        if parent.layer >= len(weights):
            continue
            
        W = weights[parent.layer]
        b = biases[parent.layer]
        
        # Pre-calculate Parent's Affine Output: z = W(Ax+c) + b
        W_hat = W @ parent.Alw
        b_hat = W @ parent.clw + b
        
        for child in parent.children:
            # --- Reconstruct Geometry from Activation ---
            q = child.activation
            s_vec = -2.0 * q + 1.0 # Map 1->-1, 0->1
            
            # 1. Local Inequalities (Dlw * x <= glw)
            # Matches logic in construction.jl
            child.Dlw = s_vec[:, None] * W_hat
            child.glw = -(s_vec * b_hat)
            
            # 2. Affine Map for Next Layer (Alw * x + clw)
            # Matches logic in construction.jl
            child.Alw = q[:, None] * W_hat
            child.clw = q * b_hat
            
            queue.append(child)
            
    return root